#Importing the libraries

In [51]:
import keras as Keras
import numpy as np 
import pandas as pd 
from hyperopt import STATUS_OK , Trials, fmin, hp , tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import mlflow 
from mlflow.models import  infer_signature 


In [52]:
data = pd.read_csv("https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv", sep=';')

In [53]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [54]:
data.shape

(4898, 12)

Split the data into train , validation  and test

In [55]:
train , test = train_test_split(data, test_size=0.25 , random_state=42)

In [56]:
train_x = train.drop(columns=["quality"], axis=1).values
train_y = train["quality"].values.ravel()

#test data
test_x = test.drop(columns=["quality"], axis=1)
test_y = test["quality"].values.ravel()
# train & validation data
train_x , valid_x , train_y , valid_y = train_test_split(train_x, train_y , test_size=0.20 , random_state=42)


In [57]:
infer_signature(train_x, train_y)

inputs: 
  [Tensor('float64', (-1, 11))]
outputs: 
  [Tensor('int64', (-1,))]
params: 
  None

In [58]:
#ANN Model 


def train_model(params, epochs , train_x, train_y , valid_x, valid_y ,test_x, test_y ):

    #define model architecture 
    mean = np.mean(train_x, axis=0)
    var = np.var(train_x, axis=0)

    model = Keras.Sequential(
        [
            Keras.Input([train_x.shape[1]]),
            Keras.layers.Normalization(mean=mean, variance=var),
            Keras.layers.Dense(64, activation="relu"),
            Keras.layers.Dense(1)
        ]
    )

    ## Compile the model 
    model.compile(optimizer=Keras.optimizers.SGD(
        learning_rate= params["lr"],
        momentum = params["momentum"]

    ),
    loss="mean_squared_error",
    metrics=[Keras.metrics.RootMeanSquaredError()]
    )

    #Train the ANN model with lr and momentum params with MLFLOW training 
    with mlflow.start_run(nested=True):
        model.fit(train_x, train_y , validation_data=(valid_x,valid_y), epochs=epochs , batch_size=64 )

        ## Evaluate the model 

        eval_result = model.evaluate(valid_x, valid_y, batch_size=64)

        eval_rsme = eval_result[1]

        ## log the parameters and results

        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse", eval_rsme)

        #Log the model 

        mlflow.tensorflow.log_model(model, "Model", signature=infer_signature(train_x, train_y))

        return {"loss": eval_rsme , "status": STATUS_OK , "model": model}

    

In [59]:
def objective(params):
    result = train_model(
        params, 
        epochs=3,
        train_x = train_x,
        train_y = train_y,
        valid_x = valid_x,
        valid_y = valid_y,
        test_x = test_x,
        test_y = test_y
    )

    return result


In [60]:
space={
    "lr": hp.loguniform("lr", np.log(1e-5 ), np.log(1e-2)),
    "momentum": hp.uniform("momentum", 0.0,1.0)
    
}


In [61]:
mlflow.set_experiment("/wine-quality")
with mlflow.start_run():
    # conduct the hyperparameter search using Hyperopt
    trails = Trials()
    best = fmin(
        fn =objective,
        space=space,
        algo = tpe.suggest,
        max_evals=4,
        trials=trails

    )

    # fetch the details of the best run

    best_run = sorted(trails.results , key=lambda x: x["loss"])[0]

    mlflow.log_params(best)
    mlflow.log_metric("eval_rsme", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=infer_signature(train_x, train_y))

    print(f"Best parameters: {best}")
    print(f"Best eval rsme :{ best_run["loss"]}")

  0%|          | 0/4 [00:00<?, ?trial/s, best loss=?]

Epoch 1/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 17s 385ms/step - loss: 37.3547 - root_mean_squared_error: 6.1119
44/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 35.8878 - root_mean_squared_error: 5.9904   
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 34.6821 - root_mean_squared_error: 5.8892 - val_loss: 32.4986 - val_root_mean_squared_error: 5.7008

Epoch 2/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 30.8636 - root_mean_squared_error: 5.5555
42/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 31.1154 - root_mean_squared_error: 5.5780 
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 30.3740 - root_mean_squared_error: 5.5113 - val_loss: 28.4186 - val_root_mean_squared_error: 5.3309

Epoch 3/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 28.0541 - root_mean_squared_error: 5.2966
45/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 27.4989 - root_mean_squared_error: 5.2

2026/02/19 23:45:07 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Epoch 1/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 21s 487ms/step - loss: 31.0074 - root_mean_squared_error: 5.5684
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 29.3810 - root_mean_squared_error: 5.4192   
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 27.1525 - root_mean_squared_error: 5.2108 - val_loss: 22.2825 - val_root_mean_squared_error: 4.7204

Epoch 2/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 21.3240 - root_mean_squared_error: 4.6178
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 18.3813 - root_mean_squared_error: 4.2873 - val_loss: 15.0583 - val_root_mean_squared_error: 3.8805

Epoch 3/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 15.2283 - root_mean_squared_error: 3.9023
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 13.5754 - root_mean_squared_error: 3.6829 
46/46 ━

2026/02/19 23:45:29 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



...

Epoch 1/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 15s 337ms/step - loss: 33.0119 - root_mean_squared_error: 5.7456
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 26.5547 - root_mean_squared_error: 5.1370   
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 20.0703 - root_mean_squared_error: 4.4800 - val_loss: 9.5241 - val_root_mean_squared_error: 3.0861

Epoch 2/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 9.8714 - root_mean_squared_error: 3.1419
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5.4518 - root_mean_squared_error: 2.3349 - val_loss: 3.1761 - val_root_mean_squared_error: 1.7822

Epoch 3/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 2.8139 - root_mean_squared_error: 1.6775
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.4942 - root_mean_squared_error: 1.5793 - val_loss: 2.

2026/02/19 23:45:53 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



100%|██████████| 4/4 [00:57<00:00, 14.36s/trial, best loss: 1.4883325099945068]

2026/02/19 23:46:03 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Best parameters: {'lr': np.float64(0.0006587207802733468), 'momentum': np.float64(0.567469120381304)}
Best eval rsme :1.4883325099945068
